In [1]:
import json
import argparse
import numpy as np

In [4]:
# 파라메터를 받아서 처리하는 함수
def build_args():
  parser = argparse.ArgumentParser(description="compute similarity score")  # 유사도  계산
  parser.add_argument('--user1',dest='user1', required=True, help = 'First user')
  parser.add_argument('--user2',dest='user2', required=True, help = 'Second user')
  parser.add_argument('--json-source',dest='json_path', required=True, help = 'json data path')
  parser.add_argument('--score-type',dest='score_type', required=True, choices=['eu','pe'] ,  help = 'eu(Euclidean) or pe(Pearson)')
  return parser

In [ ]:
# dataset  dictionary
def euclidean_score(dataset, user1,user2):
  if user1 not in dataset:
    raise TypeError(f"Cannot find {user1} in the dataset")
  if user2 not in dataset:
    raise TypeError(f"Cannot find {user2} in the dataset")
  common_movies={}
  #  두 사용자가 평가한 영화를 추출
  for item in dataset[user1]:
    if item in dataset[user2]:
      common_movies[item] = 1
  # 공통된 영화가 없으면 유사성 점수가 계산되지 않음
  if len(common_movies) == 0:
    return 0
  # 두 평점간의 제곱차이를 계산, 이를 이용해서 유클리드 점수를 계산
  squard_diff = []
  for item in dataset[user1]:
    if item in dataset[user2]:
      squard_diff.append( np.square(dataset[user1][item] - dataset[user2][item]) )

  return 1 / (np.sqrt(np.sum(squard_diff)) + 1)

In [3]:
# 피어슨 상관관계 계산 함수
def person_score(dataset,user1,user2):
  if user1 not in dataset:
    raise TypeError(f"Cannot find {user1} in the dataset")
  if user2 not in dataset:
    raise TypeError(f"Cannot find {user2} in the dataset")
  common_movies={}
  #  두 사용자가 평가한 영화를 추출
  for item in dataset[user1]:
    if item in dataset[user2]:
      common_movies[item] = 1
  # 공통된 영화가 없으면 유사성 점수가 계산되지 않음
  num_ratings = len(common_movies)
  if num_ratings == 0:
    return 0
  # 두 사용자가 평가한 모든 영화의 평점의 합
  user1_sum = np.sum([ dataset[user1][item] for item in common_movies ])
  user2_sum = np.sum([ dataset[user2][item] for item in common_movies ])
  # 두 사용자가 평가한 모든 영화의 평점의 제곱 합
  user1_squared_sum = np.sum([ np.square(dataset[user1][item]) for item in common_movies ])
  user2_squared_sum = np.sum([ np.square(dataset[user2][item]) for item in common_movies ])

  # 두 사용자가 평가한 모든 영화의 평점 곱의 합을 계산
  sum_of_products = np.sum([ dataset[user1][item]*dataset[user2][item] for item in common_movies])
  # 피어슨 상관관계 점수 계산
  Sxy = sum_of_products -  (user1_sum*user2_sum / num_ratings)
  Sxx = user1_squared_sum - np.square(user1_sum) / num_ratings
  Syy = user2_squared_sum - np.square(user2_sum) / num_ratings
  # 편차가 없으면 점수는 0
  if Sxx * Syy == 0:
    return 0
  # 피어슨 점수를 반환
  return Sxy / np.sqrt(Sxx * Syy)

In [ ]:
if __name__ == '__main__':
  args = build_args().parse_args()
  user1 = args.user1
  user2 = args.user2
  json_path = args.json_path
  score_type = args.score_type
  # 유사성 점수를 계산  
  with open(json_path,'r') as f:
    data = json.loads(f.read())
  if score_type == 'eu':
    print('Euclidean Score.....')
    print(euclidean_score(data,user1,user2))
  else:
    print('Pearson Score.....')
    print(person_score(data,user1,user2))